In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import os
import numpy as np
import gc
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [ ]:
video_folder = '/kaggle/input/surgical/11'
video_files = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith('.mp4')]
label_dict = {
    'tool_video_o4.mp4': 'label1',
}
frames = []
labels = []
for video_file in video_files:
    if video_file.split('/')[-1] in label_dict:
        label = label_dict[video_file.split('/')[-1]]
    else:
        label = 'unknown'
    text_file = video_file.replace('.mp4', '.txt')
    with open(text_file, 'r') as f:
        lines = f.readlines()
    instruments = [line.strip() for line in lines]
    cap = cv2.VideoCapture(video_file)

    if not cap.isOpened():
        print('Error opening video file:', video_file)
    else:
        while True:
            ret, frame = cap.read()

            if not ret:
                break

            frame = cv2.resize(frame, (224, 224))
            frames.append(frame)

            label_vec = np.zeros(2)
            if label == 'label1':
                label_vec[0] = 1
            else:
                label_vec[1] = 1
                
            instruments_array = np.zeros(7)
#             for i in range(len(instruments)):
            for i in range(len(instruments)):
                if instruments[i] == 'grasper':
                    instruments_array[i] = 0
                elif instruments[i] == 'Bipolar':
                    instruments_array[i] = 1
                elif instruments[i] == 'hook':
                    instruments_array[i] = 2
                elif instruments[i] == 'scissors':
                    instruments_array[i] = 3
                elif instruments[i] == 'clipper':
                    instruments_array[i] = 4
                elif instruments[i] == 'irrigator':
                    instruments_array[i] = 5
                elif instruments[i] == 'specimenbag':
                    instruments_array[i] = 6

            
            label_and_instruments = np.concatenate((label_vec, instruments_array))
            labels.append(label_and_instruments)

        cap.release()

gc.collect()
num_frames = len(frames)


In [ ]:
X_train = np.array(frames)
y_train = np.array(labels)

In [ ]:
train_data = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
)

In [ ]:
# cnn-1
model = Sequential()
model.add(Conv2D(2, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Flatten())
# model.add(Dense(2, activation='softmax'))
model.add(Dense(9, activation='softmax'))

In [ ]:
# cnn-2
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(9, activation='sigmoid'))

In [ ]:
# lenet
model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(224, 224, 3)))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh'))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))
model.add(Dense(9, activation='softmax'))


In [ ]:
# zfnet
model = Sequential()
model.add(Conv2D(96, kernel_size=(7, 7), strides=(2, 2), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(256, kernel_size=(5, 5), strides=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(7, activation='softmax'))



In [ ]:
# googlenet
model = Sequential()
model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))



In [ ]:
# alexnet
model = Sequential()
model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3), padding='valid'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


In [ ]:
# VGG
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))


In [ ]:
# polynet
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Concatenate, GlobalAveragePooling2D, Dense

def conv_block(inputs, filters, kernel_size, strides=(1, 1), padding='same'):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding, activation='relu')(inputs)
    x = Conv2D(filters, kernel_size, strides=(1, 1), padding=padding, activation='relu')(x)
    return x

def inception_block(inputs, filters):
    # Path 1
    x1 = conv_block(inputs, filters, (1, 1))

    # Path 2
    x2 = conv_block(inputs, filters, (1, 1))
    x2 = conv_block(x2, filters, (3, 3))

    # Path 3
    x3 = conv_block(inputs, filters, (1, 1))
    x3 = conv_block(x3, filters, (3, 3))
    x3 = conv_block(x3, filters, (3, 3))

    # Path 4
    x4 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    x4 = conv_block(x4, filters, (1, 1))

    # Concatenate paths
    x = Concatenate()([x1, x2, x3, x4])
    return x

inputs = Input(shape=(224, 224, 3))

x = conv_block(inputs, 32, (3, 3), strides=(2, 2))
x = conv_block(x, 64, (3, 3))
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

x = inception_block(x, 32)
x = inception_block(x, 64)
x = inception_block(x, 128)
x = inception_block(x, 256)
x = inception_block(x, 512)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1000, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
gc.collect()


In [ ]:
train_generator = train_data.flow(X_train, y_train, batch_size=2)
history = model.fit(train_generator, steps_per_epoch=len(X_train)//2, epochs=5)
# print('Training accuracy:', history.history['accuracy'])

In [ ]:
test_video_file = '/kaggle/input/surgical/train/tool_video_04.mp4'
test_text_file = '/kaggle/input/surgical/train/tool_video_04.txt'
test_frames = []
test_cap = cv2.VideoCapture(test_video_file)
if not test_cap.isOpened():
    print('Error opening test video file:', test_video_file)
else:
    while True:
        ret, frame = test_cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        test_frames.append(frame)
    test_cap.release()

with open(test_text_file, 'r') as f:
    lines = f.readlines()
test_instruments = [line.strip() for line in lines]

test_frames = np.array(test_frames)
test_instruments_array = np.zeros(7)
# for i in range(len(test_instruments)):
#     if test_instruments[i] == 'grasper':
#         test_instruments_array[0] = 1
#     elif test_instruments[i] == 'Bipolar':
#         test_instruments_array[1] = 1
#     elif test_instruments[i] == 'hook':
#         test_instruments_array[2] = 1
#     elif test_instruments[i] == 'scissors':
#         test_instruments_array[3] = 1
#     elif test_instruments[i] == 'clipper':
#         test_instruments_array[1] = 1
#     elif test_instruments[i] == 'irrigator':
#         test_instruments_array[1] = 1
#     elif test_instruments[i] == 'specimenbag':
#         test_instruments_array[1] = 1
for i in range(len(test_instruments)):
        if test_instruments[i] == 'grasper':
            test_instruments_array[i] = 0
        elif test_instruments[i] == 'Bipolar':
            test_instruments_array[i] = 1
        elif test_instruments[i] == 'hook':
            test_instruments_array[i] = 2
        elif test_instruments[i] == 'scissors':
            test_instruments_array[i] = 3
        elif test_instruments[i] == 'clipper':
            test_instruments_array[i] = 4
        elif test_instruments[i] == 'irrigator':
            test_instruments_array[i] = 5
        elif test_instruments[i] == 'specimenbag':
            test_instruments_array[i] = 6
        


test_labels = np.tile(np.concatenate((np.zeros(1), test_instruments_array)), (len(test_frames), 1))


In [ ]:
test_predictions = model.predict(test_frames)


In [ ]:
# predicted_labels = np.argmax(test_predictions[:, :2], axis=1)
# true_labels = np.argmax(test_labels[:, :2], axis=1)
# correct_predictions = np.sum(predicted_labels == true_labels)
# total_predictions = len(true_labels)
# accuracy = correct_predictions / total_predictions

# print('Test accuracy:', accuracy*100)

In [ ]:
# test_predictions = model.predict(test_frames)
predicted_labels = np.argmax(test_predictions[:, :2], axis=1)
predicted_instruments = np.argmax(test_predictions[:, 2:], axis=1) + 1  
true_labels = np.argmax(test_labels[:, :2], axis=1)
true_instruments = np.argmax(test_labels[:, 2:], axis=1) + 1  

correct_predictions = np.sum(predicted_labels == true_labels) + np.sum(predicted_instruments == true_instruments)
total_predictions = len(true_labels) + len(true_instruments)
accuracy = correct_predictions / total_predictions

print('Test accuracy:', accuracy*100)


In [ ]:
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
conf_matrix = confusion_matrix(true_instruments, predicted_instruments)

# Calculate the accuracy for each instrument class
accuracy = {}
for i in range(len(conf_matrix)):
    accuracy[i+1] = conf_matrix[i][i] / sum(conf_matrix[i])
    print(f'Accuracy for instrument class {i+1}: {accuracy[i+1]:.2f}')
    
    
import numpy as np

unique_classes = np.unique(true_instruments)
print(unique_classes)


In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['Video File', 'Instrument Prediction'])

for i in range(len(test_frames)):
    video_file = test_video_file.split('/')[-1]
    
    # Extract the predicted instrument label
    predicted_instrument = predicted_instruments[i]
    
    # Map the predicted label to its corresponding instrument name
    if predicted_instrument == 1:
        instrument_name = 'Bipolar'
    elif predicted_instrument == 2:
        instrument_name = 'Clipper'
    elif predicted_instrument == 3:
        instrument_name = 'Grasper'
    elif predicted_instrument == 4:
        instrument_name = 'Hook'
    elif predicted_instrument == 5:
        instrument_name = 'Irrigator'
    elif predicted_instrument == 6:
        instrument_name = 'Scissors'
    elif predicted_instrument == 7:
        instrument_name = 'Specimen Bag'
    else:
        instrument_name = 'Unknown'

    df = df.append({'Video File': video_file, 'Instrument Prediction': instrument_name}, ignore_index=True)
df.to_excel('instrument_predictions.xlsx', index=False)


In [ ]:
# get a random frame and its corresponding predicted instrument
idx = np.random.randint(0, len(test_frames))
frame = test_frames[idx]
predicted_instrument = predicted_instruments[idx]

instruments_dict = {
    1: 'grasper',
    2: 'Bipolar',
    3: 'hook',
    4: 'scissors',
    5: 'clipper',
    6: 'irrigator',
    7: 'specimenbag'
}
if predicted_instrument in instruments_dict:
    predicted_instrument_name = instruments_dict[predicted_instrument]
else:
    predicted_instrument_name = 'unknown'

font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (0, 255, 0)
thickness = 2
frame_with_text = cv2.putText(frame, predicted_instrument_name, org, font, fontScale, color, thickness, cv2.LINE_AA)

cv2.imwrite('random_frame_with_predicted_instrument.jpg', frame_with_text)
